# Spectral decomposition

This uses `xarray` to help keep track of the various dimensions in this multi-dimensional problem.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import xarray as xr
import segyio
import numpy as np


with segyio.open('../data/F3_16-bit_int.sgy') as s:
    seismic = segyio.cube(s)
        
i, x, t = map(np.arange, seismic.shape)

data = xr.DataArray(seismic,
                    name='amplitude',
                    coords=[i, x, t*0.004],
                    dims=['inline', 'xline', 'twt'],
                   )

These axes are xline, inline, time.

In [ ]:
data[100].T.plot.imshow(origin='upper')

## Do spectral decomposition

We'll use scipy's `spectrogram` and put the result in an `xarray.DataArray()` for convenience.

We can compute the entire spectrogram, a 4D hypercube. Then we can take whatever slices we want from there.

In [ ]:
import scipy.signal as ss

fs = 250  # Hz
window = 0.2  # seconds

nperseg = int(fs * window)  # samples in window
step = 1
noverlap = nperseg - step

# Entire cube uses too much memory.
data_ = data[50:150, 600:]

f, t, Sxx = ss.spectrogram(data_,
                           fs=fs,
                           nperseg=nperseg,
                           noverlap=noverlap,
                           axis=-1,
                           scaling='spectrum',
                           mode='magnitude',
                          )

i = np.arange(data_.shape[0]) + 50
x = np.arange(data_.shape[1]) + 600

sd = xr.DataArray(np.sqrt(Sxx),
                  name='amplitude',
                  coords=[i, x, f, t],
                  dims=['inline', 'xline', 'freq', 'time']
                 )

In [ ]:
sd.shape

In [ ]:
f

The shape is xline, inline, frequency, time. There are only 26 frequency slices because of the length of the window and the 125 Hz Nyquist limit. The frequency samples are at intervals of 1 / `window` = 5 Hz. The first two are likely garbage (out of band) and so is anything above 0.8 * Nyquist = 100 Hz, typically (there's usually a hi-cut there in seismic acquisition, to reduce aliasing).

In [ ]:
sd.coords

In [ ]:
sd[50, 100].T.plot.imshow(origin='upper')

This is a time-frequency 'spectrogram' of 1 trace, but this is not usually what you want.

## RGB display

Make a 3-channel cube for 3 frequencies (and move freq 'channels' to last axis).

This is really nice becase `xarray` will do the interpolation for us. So even if the window was a weird length and the frequency samples were not whole numbers, `xarray` can do the interpolation for us. Cool!

In [ ]:
rgb_ = sd.interp(freq=[10, 20, 40]).transpose(..., 'freq')

In [ ]:
rgb_.shape

Normalize:

In [ ]:
rgb = (rgb_ - rgb_.min()) / (rgb_.max() - rgb_.min())

And `xarray` uses `imshow` to plot a 3D array as RGB:

In [ ]:
rgb.sel(time=0.5).plot.imshow()

### EXERCISE

Can you make a plot showing a line through the seismic on the left and the same line through the RGB cube on the right?

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(15,6), sharey=True)

L = 50
data_.sel(inline=L).T.plot.imshow(ax=ax0, origin='upper')
rgb.sel(inline=L).T.plot.imshow(ax=ax1, origin='upper')

It's probably going to be a good idea to do a whitening step (per trace normalization, basically).

## Tuning cubes

Alternatively, we can get an entire cube at a particular frequency, again using `xarray`'s interpolation.

In [ ]:
tc_40Hz = sd.interp(freq=40)
tc_40Hz.shape

In [ ]:
import matplotlib.pyplot as plt

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(15,6), sharey=True)

data_[5].T.plot.imshow(ax=ax0, origin='upper')
tc_40Hz[5].T.plot.imshow(ax=ax1, origin='upper')

---

© 2021 Agile Scientific